# Wzorce projektowe

## Wstęp

Jednym z pierwszych przykładów stosowania wzorców projektowych, zanim weszły one do kanonu Informatyki, były prototypy mostów. Ze względu na lekką budowę w stosunku do ciężaru, który muszą wytrzymać oraz różne warunki pogodowe, które nie mogą mieć negatywnego wpływu na ich wytrzymałość, zbudowanie mostu nie jest łatwym zadaniem. Architekci w tym celu wykorzystują zaprojektowane i przetestowane modele mostów dopasowanych do konkretnych uwarunkowań terenu, na którym mają stać. Każdy kolejny most jest kopią istniejącego wzorca, który bez znaczących odchyleń od norm budowlanych spełnia swoje funkcje.

<center>


</center>

Czym jest wzorzec projektowy?

> Wzorzec to sprawdzona koncepcja, która opisuje problem powtarzający się wielokrotnie w określonym kontekście, działające na niego siły, oraz podaje istotę jego rozwiązania w sposób abstrakcyjny.
>
> Christopher Alexander

Podobną koncepcję zastosowano na potrzeby budowy oprogramowania. Należy poczynić w tym miejscu dwie uwagi,

* niestosowanie wzorców jest złym nawykiem,
* wykorzystywanie ich nadmiarowo bądź niepotrzebnie jest jeszcze gorszym nawykiem.

Ze względu na typy wzorców można je podzielić na:
<ol>
<li>konstrukcyjne (kreacyjne)</li>
    <ol>
    <li>opisują sposoby tworzenia obiektów,
    <li>oddzielają tworzenie od pozostałej części systemu,</li>
    </ol>
<li>strukturalne</li>
    <ol>
    <li>opisują konstrukcję obiektów,
    <li>korzystają z dziedziczenia i delegacji,</li>
    </ol>
<li>operacyjne (behawioralne)</li>
    <ol>
    <li>charakteryzują sposób interakcji między obiektami oraz przydział odpowiedzialności,</li>
    </ol>
</ol>

Zanim zostaną omówione poszczególne kategorie wzorców wraz z przykładami, zostanie przedstawiony ogólny sposób modelowania klas UML (ang. Unified Modeling Language).

## UML

UML to graficzny język do wizualizacji, definiowania, tworzenia i dokumentowania elementów systemów informatycznych. Poniżej zostaną opisane podstawowe informacje na temat języka UML w kontekście definiowania diagramów klas.

### Dziedziczenie

<center>

</center>

<ol>
<li>Obiektem nadrzędnym (`BaseClass`):</li>
<ol>
<li>klasa,</li>
<li>klasa abstrakcyjna,</li>
<li>interfejs,</li>
</ol>
<li>obiektem podrzędnym (`Extended Class`) zwykle jest klasa,</li>
<li>w niektórych języki programowania nie ma dziedziczenia wielokrotnego (diamond problem).
</li>
</ol>

Poniższy kod przedstawia sposób użycia dziedziczenia podanego na diagramie powyżej.

In [ ]:
public class BaseClass
{

}

public class ExtendedClass: BaseClass
{
    
}

### Asocjacja

<center>

</center>

1. Określa „luźne” powiązania między obiektami, oba istnieją niezależnie od siebie.
2. Obiekty nie są powiązane ze sobą na stałe.
3. Pozwala określać liczebność.

Przykład użycia tej relacji znajduje się poniżej.

In [ ]:
public class Editor
{
    public void Save() => Console.WriteLine("Saving file");
}
public class Application
{
    private Editor EditorInstance { get; set; } = new Editor();
    
    public void Exit() 
    {
        Console.WriteLine("Application Exit");
        EditorInstance.Save();
    }
}

### Agregacja

<center>

</center>

1. Szczególny (silniejszy) przypadek asocjacji wyrażający zależność typu zawiera.
2. Agregacja jest często przedstawiana w opozycji do dziedziczenia.
3. `Katalog` zawiera `Książki`, `Książka` może należeć do wielu `Katalogów`. Istnienie bądź nieistnienie instancji `Książka` i instancji `Katalog` nie wpływa na siebie.

In [ ]:
using System.Collections.Generic;

public class Book
{
    public string Title { get; set; }
    public string Author { get; set; }
}

public class Catalog
{
    private List<Book> BookList { get; set; } = new List<Book>();    
    
    public List<Book> FindByTitle(string title)
    {
        return BookList.Where(x=> x.Title.Contains(title)).ToList();
    }
    
    public void AddBook(params Book[] books) => BookList.AddRange(books);
}

var book = new Book() { Title = "A Thousand Splendid Suns", Author = "Khaled Hosseini" };
var book2 = new Book() { Title = "Chasing the Devil: The Search for Africa'''s Fighting Spirit", Author = "Tim Butcher" };
var book3 = new Book() { Title = "Undeniable Series", Author = "Ramona Gray" };

Catalog library1 = new Catalog();
library1.AddBook(book, book2, book3);

Catalog library2 = new Catalog();
library2.AddBook(book2, book3);

library1.FindByTitle("A").ForEach(b => Console.WriteLine($"Found book: {book.Title}"));

Jak widać na powyższym przykładzie jedna instancja książki `Book` może należeć do jednego lub wielu księgozbioru.

### Kompozycja

<center>

</center>

1. Szczególny przypadek agregacji. Mówimy `Chapter` wchodzi w skład `Book`.
2. Część nie może istnieć bez całości, a usunięcie całości powoduje automatyczne usunięcie wszystkich jej części, związanych z nią związkiem kompozycji.
3. `Book` składa się z `Chapter`, `Chapter` jest częścią (wchodzi w skład) `Book`, `Chapter` może należeć tylko do jednej `Book`. Istnienie `Book` decyduje o istnieniu `Chapter` i na odwrót.

In [ ]:
using System.Collections.Generic;

public class Chapter
{
    public string Name { get; set; }
    public string Text { get; set; }
    
    public Chapter(string name, string text) {
        Name = name;
        Text = text;
    }
    
    public override string ToString() => $"Chapter: {Name}\n----------\n{Text}";
}

public class Book
{
    private List<Chapter> Chapters { get; set; } = new List<Chapter>();
    
    protected void AddChapters(params Chapter[] chapters) => Chapters.AddRange(chapters);
    
    public override string ToString() => String.Join("\n\n", Chapters);
}

public class ScienceBook: Book
{
    public ScienceBook() {
        base.AddChapters(
            new Chapter("Introduction", "Lorem ipsum"), 
            new Chapter("Backgroud", "Lorem ipsum")
        );
    }
}

Book scienceBook = new ScienceBook();

Console.WriteLine(scienceBook);


### Siła powiązań

* Asocjacja – luźne powiązanie,
* agregacja – obiekt agregowany dopasowany jest do obiektu agregującego (specjalistyczny), ale obiekty można wymieniać między instancjami tego samego typu,
* kompozycja – bardzo silne oddziaływanie. Brak możliwości istnienia instancji poza kompozytem.

Poniższy przykład przedstawia zależności i siłę powiązań między obiektami.

<center>


</center>

## Wzorce konstrukcyjne

### Budowniczy

Charakteryzuje się uniezależnieniem działania systemu od implementacji interfejsu.

<center>

</center>

In [ ]:
class HtmlElement
{  
    public string Name, Text;  
    public List<HtmlElement> Elements = new List<HtmlElement>();  
    private const int indentSize = 2;  
    public HtmlElement() {}  
    public HtmlElement(string name, string text)  
    {
        Name = name;
        Text = text;  
    }
}

### Singleton

Gwarantuje, że w systemie będzie tylko jedna instancja klasy. Należy pamiętać, że w ramach tworzenia wielu `AppDomain` może istnieć więcej niż jedna instancja statycznej zmiennej (zwykle aplikacja składa się z jednego `AppDomain`). Dodatkowo można wyróżnić wersję równoległą tego wzorca projektowego.

<center>

</center>

In [ ]:
public sealed class Singleton
{
    private static volatile Singleton instance;
    private static object syncRoot = new Object();
    private Singleton() {}
    public static Singleton Instance
    {
        get
        {
            if (instance == null)
            {
                lock (syncRoot)
                {
                  if (instance == null)
                       instance = new Singleton();
                }
            }
            return instance;
        }
    }
}

### Metoda wytwórcza

Określa interfejs do tworzenia obiektów, przy czym umożliwia podklasom wyznaczenie klasy danego obiektu.

<center>

<center>

### Fabryka abstrakcyjna

Udostępnia interfejs do tworzenia rodziny powiązanych ze sobą lub zależnych od siebie obiektów bez określania ich klas konkretnych.

<center>

</center>

### Fabryka abstrakcyjna i metoda wytwórcza

1. Fabryki abstrakcyjnej używamy, kiedy mówimy o rodzinach klas.
2. Metody wytwórczej używamy, kiedy tworzymy obiekt w konkretnej metodzie i zwracamy interfejs stworzonego obiektu.

### Prototyp

Określa na podstawie prototypowanego egzemplarza rodzaje tworzonych obiektów i generuje nowe obiekty przez kopiowanie tego prototypu.

<center>

</center>

In [ ]:
class Primitive
{
    public string Color { get; set; }
    
    public int X { get; set; }
    
    public int Y { get; set; }
    
    public Primitive(string color, int x, int y) {
        Color = color;
        X = x;
        Y = y;
    }
    
    public Primitive Clone() => new Primitive(this.Color, this.X, this.Y);
    
    public void Move(int x, int y) {
        this.X = x; this.Y = y;
    } 
}
class PrimitiveManager
{
    private Dictionary<string, Primitive> primitives =
        new Dictionary<string, Primitive>();

    public Primitive this[string key]
    {
      get { return primitives[key]; }
      set { primitives.Add(key, value); }
    }
}

PrimitiveManager manager = new PrimitiveManager();
manager["ball"] = new Primitive("red", 0, 0);
manager["cube"] = new Primitive("green", 2, 2);

Primitive b1 = manager["ball"].Clone() as Primitive;
b1.Move(-1,1);
b1.Color = "blue";

Primitive c1 = manager["cube"].Clone() as Primitive;

## Wzorce strukturalne

### Adapter (Wrapper)

Podobny do adaptera mini-jack – jack. Sygnał płynie ten sam, a zmianie ulga jedynie interfejs.

<center>

</center>

### Dekorator (Decorator)

Dynamicznie dołącza dodatkowe obowiązku do obiektu. Wzorzec ten udostępnia alternatywny i elastyczny sposób tworzenia podklas o wzbogacone funkcje oraz zmniejsza ich liczbę (bardzo dużo kombinacji a mało typów).

<center>

</center>

Przykład użycia

<center>

</center>

### Fasada (Facade)

Udostępnia jednolity interfejs dla zbioru interfejsów z podsystemu. Fasada określa interfejs wyższego poziomu ułatwiający korzystanie z podsystemów.

<center>


</center>

### Kompozyt (Composite)

Składa obiekty w struktury drzewiaste odzwierciedlające hierarchię typu część całość. Wzorzec umożliwia klientom traktowanie poszczególnych obiektów i ich złożeń w taki sam sposób.

<center>

</center>

### Most (Bridge)

Oddziela abstrakcję od jej implementacji, dzięki czemu można modyfikować te dwa elementy niezależnie od siebie.

<center>

</center>

Implementacja:

In [ ]:
public interface ILogWriter
{
    void SaveTo(string message);
}

public class DatabaseLog: ILogWriter
{
    public void SaveTo(string message)
    {
        Console.WriteLine($"DatabaseLog: {message}");
    }
}

public class SystemLogEvent: ILogWriter
{
    public void SaveTo(string message)
    {
        Console.WriteLine($"SystemLogEvent: {message}");
    }
}

public interface ILogger
{
    ILogWriter Writer { get; set; }
    void Write(string message);
}

public class ServiceLogger: ILogger
{
    public ILogWriter Writer { get; set; }
    
    public void Write(string message) {
        Writer.SaveTo(message);
    }
}

ILogger log = new ServiceLogger();

log.Writer = new DatabaseLog();
log.Write("Message 1");
//DB not responding
log.Writer = new SystemLogEvent();
log.Write("Message 2");

### Pełnomocnik (Proxy)

Udostępnia zastępnik lub reprezentanta innego obiektu w celu kontrolowania dostępu do niego.

<center>

</center>

### Pyłek (Flyweight)

Wykorzystuje współdzielenie do wydajnej obsługi dużej liczby małych obiektów.

<center>

</center>

## Wzorce operacyjne (behawioralne)

### Interpreter

Określa reprezentację gramatyki języka oraz interpreter, który wykorzystuje tę reprezentację do interpretowania zdań z danego języka.

<center>

</center>

In [ ]:
string roman = "MCMXXVIII";
Context context = new Context(roman);

List<Expression> tree = new List<Expression>();
tree.Add(new ThousandExpression());
tree.Add(new HundredExpression());
tree.Add(new TenExpression());
tree.Add(new OneExpression());

foreach (Expression exp in tree)
{
    exp.Interpret(context);
}

### Iterator

Zapewnia sekwencyjny dostęp do elementów obiektu złożonego bez ujawniania jego wewnętrznej reprezentacji.

<center>

</center>

### Łańcuch zobowiązań (Chain of Responsibility)

Pozwala uniknąć nadawcy żądania z jego odbiorcą, ponieważ umożliwia obsłużenie żądania więcej niż jednemu obiektowi. Łączy w łańcuch obiekty odbiorcze i przekazuje między żądanie do momentu obsłużenia go.

<center>

</center>

In [ ]:
IHandler EmailLog;
IHandler DatabaseLog;
IHandler FileLog;
IHandler DummyLog;
…
EmailLog.SetNext(DatabaseLog);
DatabaseLog.SetNext(FileLog);
FileLog.SetNext(DummyLog);
…
EmailLog.ProcessRequest(req);

### Mediator

Określa obiekt kapsułkujący informacje o interakcji między obiektami danego zbioru. Wzorzec ten pomaga zapewnić luźne powiązanie, ponieważ zapobiega bezpośredniemu odwoływaniu się obiektów do siebie i umożliwia niezależne modyfikowanie interakcji między nimi.

<center>

</center>

Implementacja:

<center>

</center>

In [ ]:
ChatRoom room = new StudentChatRoot();

User u1 = new User1(m);
User u2 = new User2(m);

Room.AddUsers(u1,u2);

u1.Send("How are you?");
u2.Send("Fine, thanks");
…
public ChatRoom.Send(Message mess)
{
    foreach(User u in Users) {
        if( u != mess.Sender)
            u.Send(mess);
    }
}



### Obserwator (Observer)

Określa zależność jeden do wielu między obiektami. Kiedy zmieni się stan jednego z obiektów wszystkie obiekty zależne od niego są o tym automatycznie powiadamiane i aktualizowane.

<center>

</center>

### Odwiedzający (Visitor)

Prezentuje operację wykonywaną na elementach struktury obiektów. Wzorzec ten umożliwia nowej operacji bez zmieniania klas elementów, na których działa.

<center>

</center>

Implementacja

<center>

</center>

In [ ]:
public class HtmlVisitor : IVisitor
{
    public string Out = „”;

    public void Visit(BoldText part)
    {
        Out += "<b>" + part.Text + "</b>";
    }
    public void Visit(Hyperlink part)
    {
        Out += "<a href=\"" + part.Url + "\">" + part.Text + "</a>";
    }
}

Document doc = new Document(
    new BoldFont(„text”),
    new BoldFont(„text2”),
    new Hyperlink(„UŚ”, „us.edu.pl”))
HtmlVisitor visitor = new HtmlVisitor();
string ouput = doc.Visit(visitor);

### Pamiątka (Memento)

Bez naruszania kapsułkowania rejestruje i zapisuje w wewnętrznej jednostce wewnętrzny stan obiektu, co umożliwia późniejsze przywrócenie obiektu według zapamiętanego stanu.

In [ ]:
SalesProspect s = new SalesProspect();
s.Name = "Noel van Halen";
s.Phone = "(412) 256-0990";
s.Budget = 25000.0;

ProspectMemory m = new ProspectMemory();
m.Memento = s.SaveMemento();

s.Name = "Leo Welch";
s.Phone = "(310) 209-7111";
s.Budget = 1000000.0;

s.RestoreMemento(m.Memento);

### Polecenie (Command)

Kapsułkuje żądanie w formie obiektu. Umożliwia to parametryzację klienta przy użyciu rożnych żądań w kolejkach i dziennikach, a także zapewania obsługę cofania operacji (przy użyciu wzorca pamiątka). Przykładem zastosowania jest w WPF.

<center>

</center>

### Strategia

Określa rodzinę algorytmów, kapsułkuje każdy z nich i umożliwia ich zmienne stosowanie. Wzorzec ten pozwala zmieniać algorytm niezależnie od korzystających z nich klientów.

<center>

</center>

In [ ]:
SortedList studentRecords = new SortedList();

studentRecords.Add("Samual");
studentRecords.Add("Jimmy");
studentRecords.Add("Sandra");
studentRecords.Add("Vivek");
studentRecords.Add("Anna");

studentRecords.SetSortStrategy(new QuickSort());
studentRecords.Sort();

studentRecords.SetSortStrategy(new ShellSort());
studentRecords.Sort();

studentRecords.SetSortStrategy(new MergeSort());
studentRecords.Sort();

### Strategia a Most

Strategia koncentruje się na kapsułkowaniu algorytmów (wzorzec operacyjny). Natomiast Most oddziela abstrakcję od realizacji, aby zapewnić różne wykonania dla tego samego typu.

## Bibliografia

1. http://wazniak.mimuw.edu.pl/
2. http://www.ipipan.waw.pl/~subieta/artykuly/JezykUML.doc
3. http://en.wikipedia.org/
4. Erich Gamma, Wzorce projektowe. Elementy oprogramowania obiektowego wielokrotnego użytku, Helion
5. http://sourcemaking.com/design_patterns/
6. http://www.dofactory.com/net/decorator-design-pattern
7. https://msdn.microsoft.com/en-us/magazine/cc188707.aspx
8. Dmitri Nesteruk, Design Patterns in .NET Core 3: Reusable Approaches in C# and F# for Object-Oriented Software Design, Apress, 2021


## Zadania do wykonania

### Zadanie 1

Zaimplementuj trzy wybrane wzorce projektowe przedstawione powyżej.

### Zadanie 2

Zaimplementuj klasę `SortedList` ze wzorca projektowego strategia.

### Zadanie 3

Zaimplementuj klasę obserwator dla wybranej kolekcji. W momencie zmiany wartości właściwości wyświetl na konsoli informację, co zostało zmienione.

In [ ]:
//Zad1 1 Wzorzec projektowy: Builder
public interface Builder
{
    void BuildPart1();
    void BuildPart2();
    void BuildPart3();
}

public class FullBuilder : Builder
{
    private Product product = new Product();
    
    public FullBuilder()
    {
        this.Reset();
    }
    
    public void Reset()
    {
        this.product = new Product();
    }
    
    public void BuildPart1()
    {
        this.product.Add("Part1");
    }
    
    public void BuildPart2()
    {
        this.product.Add("Part2");
    }
    
    public void BuildPart3()
    {
        this.product.Add("Part3");
    }
    
    public Product GetProduct()
    {
        Product result = this.product;
        this.Reset();
        return result;
    }
}

public class Product
{
    private List<object> parts = new List<object>();

    public void Add(string part)
    {
        this.parts.Add(part);
    }
    
    public string ListParts()
    {
        string str = string.Empty;
        for (int i = 0; i < this.parts.Count; i++)
        {
            str += this.parts[i] + ", ";
        }
        str = str.Remove(str.Length - 2);
        return "Product parts: " + str + "\n";
    }
}

public class Director
{
    private Builder builder;

    public Builder Builder
    {
        set { builder = value; } 
    }
    
    public void BuildMinimalProduct()
    {
        this.builder.BuildPart1();
    }
    
    public void BuildFullProduct()
    {
        this.builder.BuildPart1();
        this.builder.BuildPart2();
        this.builder.BuildPart3();
    }
}

var director = new Director();
var builder = new FullBuilder();
director.Builder = builder;

Console.WriteLine("Basic product:");
director.BuildMinimalProduct();
Console.WriteLine(builder.GetProduct().ListParts());

Console.WriteLine("Full product:");
director.BuildFullProduct();
Console.WriteLine(builder.GetProduct().ListParts());

Console.WriteLine("Custom product:");
builder.BuildPart2();
builder.BuildPart3();
Console.Write(builder.GetProduct().ListParts());

In [4]:
// Zad2 Wzorezc projektowy:Singleton

public sealed class Singleton
{
    private static  Singleton instance = new Singleton();
    
    static Singleton() { }
    private Singleton() { }

    public static Singleton Instance
    {
        get { return instance; }
    }

    public void DoSomething()
    {
        Console.WriteLine("Singleton is doing something...");
    
    }
}
Singleton s1 = Singleton.Instance;
Singleton s2 = Singleton.Instance;
Console.WriteLine(s1 == s2);
s1.DoSomething();
s2.DoSomething();


True
Singleton is doing something...
Singleton is doing something...


In [6]:
// Zad1 3 Wzorzec projektowy Iterator

abstract class Iterator : IEnumerator
{
    object IEnumerator.Current => Current();
    public abstract int Key();
    public abstract object Current();
    public abstract bool MoveNext();
    public abstract void Reset();
}

abstract class IteratorA : IEnumerable
{
    public abstract IEnumerator GetEnumerator();
}

class Order : Iterator
{
    private Collection collection;
    private int position = -1;
    private bool reverse = false;

    public Order(Collection collection, bool reverse = false)
    {
        this.collection = collection;
        this.reverse = reverse;

        if (reverse)
        {
            this.position = collection.getItems().Count;
        }
    }

    public override int Key()
    {
        return this.position;
    }

    public override object Current()
    {
        return this.collection.getItems()[position];
    }

    public override bool MoveNext()
    {
        int updatedPosition = this.position + (this.reverse ? -1 : 1);

        if (updatedPosition >= 0 && updatedPosition < this.collection.getItems().Count)
        {
            this.position = updatedPosition;
            return true;
        }
        else
        {
            return false;
        }
    }

    public override void Reset()
    {
        this.position = this.reverse ? this.collection.getItems().Count - 1 : 0;
    }
}
abstract class IteratorAggregate : IEnumerable
    {
        public abstract IEnumerator GetEnumerator();
    }
class Collection : IteratorAggregate
{
    List<string> collection = new List<string>();
    bool direction = false;

    public List<string> getItems()
    {
        return collection;
    }

    public void AddItem(string item)
    {
        this.collection.Add(item);
    }

    public void ReverseDirection()
    {
        direction = !direction;
    }

    public override IEnumerator GetEnumerator()
    {
        return new Order(this, direction);
    }
}

var collection = new Collection();
collection.AddItem("A");
collection.AddItem("B");
collection.AddItem("C");
collection.AddItem("D");

foreach (var element in collection)
{
    Console.WriteLine(element);
}

Console.WriteLine();
collection.ReverseDirection();

foreach (var element in collection)
{
    Console.WriteLine(element);
}

A
B
C
D

D
C
B
A


In [7]:
// Zad2

class SortedList
{
    private Strategy strategy;
    private List<String> list = new List<String>();
    public SortedList() {}

    public SortedList(Strategy strategy)
    {
        this.strategy = strategy;
    }

    public void SetSortStrategy(Strategy strategy)
    {
        this.strategy = strategy;
    }

    public void Sort()
    {
        list = strategy.DoAlgorithm(list);
        Console.Write("\n{0}: ", strategy.GetType().Name);
        list.ForEach(item => Console.Write("{0}, ", item));

    }
    public void Add(String value)
    {
        list.Add(value);
    }

}

public interface Strategy
{
    List<String> DoAlgorithm(List<String> data);
}

class QuickSort : Strategy
{
    public List<String> DoAlgorithm(List<String> a)
    {
        if (a.Count()<2)
        {
            return a;
        }
        else
        {
            string pivot = a[a.Count()/2];
            List<string> less = new List<string>();
            List<string> greater = new List<string>();
            foreach (string item in a)
            {
                for (int i=0; i<item.Count(); i++)
                {
                    if (item[i]>pivot[i])
                    {
                        greater.Add(item);
                        break;
                    }
                    if (item[i]<pivot[i])
                    {
                        less.Add(item);
                        break;
                    }
                }
            }
            List<string> temp = new List<string>();
            temp.AddRange(DoAlgorithm(less));
            temp.Add(pivot);
            temp.AddRange(DoAlgorithm(greater));
            return temp;
        }
    }
}

class ShellSort : Strategy
{
    public List<String> DoAlgorithm(List<String> a)
    {
        if (a.Count()<2)
        {
            return a;
        }
        else
        {
            for (int interval = a.Count()/2; interval>0; interval/=2)
            {
                for (int i=interval; i<a.Count(); i++)
                {
                    var temp=a[i];
                    int j;
                    for (j=i; j>=interval && a[j-interval][0]>temp[0]; j-=interval)
                    {
                        a[j]=a[j-interval];
                    }
                    a[j]=temp;
                }
            }
            return a;
        }
    }
}

class MergeSort : Strategy
{
    public List<String> DoAlgorithm(List<String> a)
    {
        if (a.Count()<2)
    {
        return a;
    }
    else
    {
        int mid = a.Count()/2;
        List<string> l = new List<string>();
        List<string> r = new List<string>();

        for (int i=0; i<a.Count(); i++)
        {
            if (i<mid)
            {
                l.Add(a[i]);
            }
            if (i>=mid)
            {
                r.Add(a[i]);
            }
        }
        merge(a, DoAlgorithm(l), DoAlgorithm(r), mid, a.Count()-mid);
        return a;
    }
    }

    public void merge(List<string> a, List<string> l, List<string> r, int left, int right)
    {   
        int i=0, j=0, k=0;
        while (i<left && j<right)
        {
            if (l[i][0]<=r[j][0])
            {
                a[k++]=l[i++];
            }
            else
            {
                a[k++]=r[j++];
            }
        }
        while (i<left)
        {
            a[k++] = l[i++];
        }
        while (j<right)
        {
            a[k++] = r[j++];
        }
    }
}


SortedList studentRecords = new SortedList();

studentRecords.Add("Szymon");
studentRecords.Add("Marcin");
studentRecords.Add("Berta");
studentRecords.Add("Lukasz");
studentRecords.Add("Zofia");

studentRecords.SetSortStrategy(new QuickSort());
studentRecords.Sort();

studentRecords.SetSortStrategy(new ShellSort());
studentRecords.Sort();

studentRecords.SetSortStrategy(new MergeSort());
studentRecords.Sort();



QuickSort: Berta, Lukasz, Marcin, Szymon, Zofia, 
ShellSort: Berta, Lukasz, Marcin, Szymon, Zofia, 
MergeSort: Berta, Lukasz, Marcin, Szymon, Zofia, 

In [1]:
// Zad3

public interface Observer
{
    void Update(ISubject subject);
}

public interface ISubject
{
    void Attach(Observer observer);
    void Detach(Observer observer);
    void Notify();
}

public class Subject : ISubject
{
    public int state { get; set; } = 0;
    private List<Observer> observers = new List<Observer>();

    public void Attach(Observer observer)
    {
        Console.WriteLine("Subject: Attached {0}", observer.GetType().Name);
        this.observers.Add(observer);
    }

    public void Detach(Observer observer)
    {
        Console.WriteLine("Subject: Detached {0}", observer.GetType().Name);
        this.observers.Remove(observer);
    }

    public void Notify()
    {
        foreach (var observer in observers)
        {
            observer.Update(this);
        }
    }
}

class ObserverA : Observer
{
    public void Update(ISubject subject)
    {
        if ((subject as Subject).state <= 3)
        {
            Console.WriteLine("ObserverA: Reacted to event");
        }
        else
        {
            Console.WriteLine("ObserverA: Not reacted to event");
        }
    }
}

class ObserverB : Observer
{
    public void Update(ISubject subject)
    {
        if ((subject as Subject).state > 3)
        {
            Console.WriteLine("ObserverB: Reacted to event");
        }
        else
        {
            Console.WriteLine("ObserverB: Not reacted to event");
        }
    }
}

var subject = new Subject();
var obsA = new ObserverA();
var obsB = new ObserverB();

subject.Attach(obsA);
subject.Attach(obsB);
Console.WriteLine();

subject.state = 1;
subject.Notify();
Console.WriteLine();

subject.state = 5;
subject.Notify();
Console.WriteLine();

subject.Detach(obsB);
Console.WriteLine();

subject.state = 3;
subject.Notify();
Console.WriteLine();

subject.Detach(obsA);
Console.WriteLine();

Subject: Attached ObserverA
Subject: Attached ObserverB

ObserverA: Reacted to event
ObserverB: Not reacted to event

ObserverA: Not reacted to event
ObserverB: Reacted to event

Subject: Detached ObserverB

ObserverA: Reacted to event

Subject: Detached ObserverA

